In [1]:
import pandas as pd
import re

## 1. Algorithm

In [2]:
# solution
def word_find_exact_match(line,words):

    found = []
    
    for to_be_searched in words:
        if re.search(r'\b' + to_be_searched + r'\b', line):
            found.append(to_be_searched)

    return found

In [3]:
# algorithm that gets all the keywords present in the text (and returns it)
def get_keywords_present(interest_string: str, keywords: list) -> tuple([bool, list]):

    # get the keywords
    keywords_present = word_find_exact_match(interest_string.lower(), keywords)

    # return a boolean to say if there is a match or not
    are_keywords_present = not len(keywords_present) == 0
    
    return are_keywords_present, keywords_present

In [4]:
keywords_db = pd.read_excel("lobbying_climate_keywords.xlsx", sheet_name="final list", engine="openpyxl")
keywords_db = keywords_db[['Original keywords', "synonyms/related"]]
keywords_db = keywords_db.fillna("")
keywords_db.head(2)

,Original keywords,synonyms/related
0,climate change,
1,global warming,


In [5]:
# create a list that will contain all the keywords on python
keywords = []

for i in keywords_db.index:
    # get the keywords on the first column
    keywords.append(keywords_db.iloc[i]['Original keywords'])

    if not keywords_db.iloc[i]['synonyms/related'] == "":
        keywords.extend([x.strip() for x in keywords_db.iloc[i]['synonyms/related'].split(',')])

# get all the keywords to be lower
for i in range(len(keywords)):
    keywords[i] = keywords[i].lower()

keywords[0:10]

['climate change',
 'global warming',
 'greenhouse',
 'ghg',
 'ghgs',
 'greenhouse gases',
 'keystone',
 'renewable energy',
 'renewables',
 'renewable energies']

## 2. Run analysis on meetings

In [18]:
#company = "ExxonMobil"
#company = "Shell"
#company = "Volkswagen"
#company = "Amazon"
company = "Uniper"

In [19]:
def run_analysis(company: str, keywords: list) -> pd.DataFrame:
    # load data
    meetings = pd.read_csv("Fossil fuel case/" + company + "_meetings.csv")
    meetings = meetings[['Date','Subject']].copy()

    # run analysis
    meetings['Relevant'] = ""
    meetings['Keywords found'] = ""

    for i in meetings.index:
    # filter the interests
        are_keywords_present, keywords_present = get_keywords_present(meetings.iloc[i]["Subject"], keywords)
        meetings.at[i, 'Relevant'] = are_keywords_present
        meetings.at[i, 'Keywords found'] = keywords_present

    print("% of relevant meetings: " + str(meetings.loc[meetings["Relevant"] == True].shape[0]/meetings.shape[0]))

    meetings.to_excel("Fossil fuel case/final filtering/" + company + "_meetings_filtered.xlsx")

    return meetings

In [20]:
meetings = run_analysis(company, keywords)

% of relevant meetings: 0.1


In [21]:
meetings.head(10)

,Date,Subject,Relevant,Keywords found
0,2016-11-28,digital agenda and energy union,False,[]
1,2017-03-23,CAN GAS SPUR THE ENERGY TRANSITION IN CENTRAL ...,False,[]
2,2021-10-21,Europe´s hydrogen economy,False,[]
3,2022-10-21,Exchange on the new Commission proposal measur...,False,[]
4,2022-10-21,Commission proposal on measures on energy pric...,False,[]
5,2022-10-26,Joint purchasing options.,False,[]
6,2023-01-19,European large-scale projects on both LNG and ...,False,[]
7,2023-01-19,European large-scale projects on both LNG and ...,False,[]
8,2023-05-04,Importance of hydropower in ensuring electrici...,True,[renewable energy]
9,2023-05-04,Presentation of new business group,False,[]
